In [23]:
from datetime import datetime, timedelta
import json
import uuid
import os
import random
from sqlalchemy import create_engine

from utils import reset_db, get_session
from data.models import cultpass

# Udahub Accounts

## Cultpass Database

**Init DB**

In [28]:
cultpass_db = "data/core/external/cultpass.db"

In [29]:
reset_db(cultpass_db)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'data/core/external/cultpass.db'

In [30]:
engine = create_engine(f"sqlite:///{cultpass_db}", echo=False)
cultpass.Base.metadata.create_all(engine)

**Experiences**

In [31]:
experience_data = []

with open('data/core/external/cultpass_experiences.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        experience_data.append(json.loads(line))

In [32]:
experience_data

[{'title': 'Carnival History Tour in Olinda',
  'description': "Discover the origins and vibrant traditions of Pernambuco's Carnival.",
  'location': 'Pernambuco, Brazil'},
 {'title': 'Sunset Paddleboarding',
  'description': 'Glide across calm waters at golden hour with all gear included.',
  'location': 'Santa Catarina, Brazil'},
 {'title': 'Pelourinho Colonial Walk',
  'description': 'Wander through colorful streets and learn about Afro-Brazilian history.',
  'location': 'Bahia, Brazil'},
 {'title': 'Samba Night at Lapa',
  'description': 'Dance the night away at a traditional samba club in the Lapa arches.',
  'location': 'Rio de Janeiro, Brazil'},
 {'title': 'Christ the Redeemer Experience',
  'description': 'Take a guided trip to one of the New Seven Wonders of the World with historical context.',
  'location': 'Rio de Janeiro, Brazil'},
 {'title': 'Modern Art at MASP',
  'description': 'Enjoy a guided visit to the São Paulo Museum of Art with insights into its top collections.',

In [33]:
with get_session(engine) as session:
    experiences = []

    for idx, experience in enumerate(experience_data):
        exp = cultpass.Experience(
            experience_id=str(uuid.uuid4())[:6],
            title=experience["title"],
            description=experience["description"],
            location=experience["location"],
            when=datetime.now() + timedelta(days=idx+1),
            slots_available=random.randint(1,30),
            is_premium=(idx % 2 == 0)
        )
        experiences.append(exp)

    session.add_all(experiences)

**User**

In [34]:
cultpass_users = []

with open('data/core/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [35]:
cultpass_users

[{'id': 'a4ab87',
  'name': 'Alice Kingsley',
  'email': 'alice.kingsley@wonderland.com',
  'is_blocked': True},
 {'id': 'f556c0',
  'name': 'Bob Stone',
  'email': 'bob.stone@granite.com',
  'is_blocked': False},
 {'id': '88382b',
  'name': 'Cathy Bloom',
  'email': 'cathy.bloom@florals.org',
  'is_blocked': False},
 {'id': '888fb2',
  'name': 'David Noir',
  'email': 'david.noir@shadowmail.com',
  'is_blocked': True},
 {'id': 'f1f10d',
  'name': 'Eva Green',
  'email': 'eva.green@ecosoul.net',
  'is_blocked': False},
 {'id': 'e6376d',
  'name': 'Frank Ocean',
  'email': 'frank.ocean@seawaves.io',
  'is_blocked': False}]

In [36]:
with get_session(engine) as session:
    # Check if users exist to avoid duplicates if cell is re-run without reset
    if session.query(cultpass.User).first() is None:
        db_users = []
        for user_info in cultpass_users:
            user = cultpass.User(
                user_id=user_info["id"],
                full_name=user_info["name"],
                email=user_info["email"],
                is_blocked=user_info["is_blocked"],
                created_at=datetime.now()
            )
            db_users.append(user)
        session.add_all(db_users) 

**Subscription**

In [37]:
with get_session(engine) as session:
    if session.query(cultpass.Subscription).first() is None:
        subscriptions = []
        for user_info in cultpass_users:
            subscription = cultpass.Subscription(
                subscription_id=str(uuid.uuid4())[:6],
                user_id=user_info["id"],
                status=random.choice(["active", "cancelled"]),
                tier=random.choice(["basic", "premium"]),
                monthly_quota=random.randint(2,10),
                started_at=datetime.now()
            )
            subscriptions.append(subscription)

        session.add_all(subscriptions)

**Reservation**

In [38]:
with get_session(engine) as session:
    if session.query(cultpass.Reservation).first() is None:
        experience_ids = [
            exp.experience_id 
            for exp 
            in session.query(cultpass.Experience).all()
        ]
        
        reservations = []
        for user_info in cultpass_users:
            # Create 0-2 reservations per user
            for _ in range(random.randint(0, 2)):
                reservation = cultpass.Reservation(
                    reservation_id=str(uuid.uuid4())[:6],
                    user_id=user_info["id"],
                    experience_id=random.choice(experience_ids),
                    status="reserved",
                )
                reservations.append(reservation)

        session.add_all(reservations)

**User Memory**

In [39]:
with get_session(engine) as session:
    if session.query(cultpass.UserMemory).first() is None:
         # Add some dummy memories for demonstration
         memories = [
             cultpass.UserMemory(memory_id=str(uuid.uuid4())[:6], user_id=cultpass_users[0]["id"], content="Prefers outdoor activities"),
             cultpass.UserMemory(memory_id=str(uuid.uuid4())[:6], user_id=cultpass_users[1]["id"], content="Interested in historical tours"),
             cultpass.UserMemory(memory_id=str(uuid.uuid4())[:6], user_id=cultpass_users[0]["id"], content="Allergic to peanuts"),
         ]
         session.add_all(memories)

# Tests

In [40]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(user)

<User(user_id='a4ab87', email='alice.kingsley@wonderland.com', is_blocked=True)>
<User(user_id='f556c0', email='bob.stone@granite.com', is_blocked=False)>
<User(user_id='88382b', email='cathy.bloom@florals.org', is_blocked=False)>
<User(user_id='888fb2', email='david.noir@shadowmail.com', is_blocked=True)>
<User(user_id='f1f10d', email='eva.green@ecosoul.net', is_blocked=False)>
<User(user_id='e6376d', email='frank.ocean@seawaves.io', is_blocked=False)>


In [41]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(f"User: {user.full_name}, Subscription: {user.subscription}")

User: Alice Kingsley, Subscription: <Subscription(subscription_id='634270', user_id='a4ab87', status='active', tier='premium')>
User: Bob Stone, Subscription: <Subscription(subscription_id='178a24', user_id='f556c0', status='cancelled', tier='premium')>
User: Cathy Bloom, Subscription: <Subscription(subscription_id='ec57d3', user_id='88382b', status='cancelled', tier='basic')>
User: David Noir, Subscription: <Subscription(subscription_id='88d986', user_id='888fb2', status='cancelled', tier='basic')>
User: Eva Green, Subscription: <Subscription(subscription_id='ffb4ab', user_id='f1f10d', status='active', tier='basic')>
User: Frank Ocean, Subscription: <Subscription(subscription_id='040211', user_id='e6376d', status='cancelled', tier='premium')>


In [42]:
with get_session(engine) as session:
    experiences = session.query(cultpass.Experience).limit(5).all()
    for experience in experiences:
        print(experience)

<Experience(experience_id='413e1f', title='Carnival History Tour in Olinda', when='2026-01-07 10:56:21.736786')>
<Experience(experience_id='e908ea', title='Sunset Paddleboarding', when='2026-01-08 10:56:21.780613')>
<Experience(experience_id='9f84c1', title='Pelourinho Colonial Walk', when='2026-01-09 10:56:21.780723')>
<Experience(experience_id='39c3e2', title='Samba Night at Lapa', when='2026-01-10 10:56:21.780773')>
<Experience(experience_id='3ede51', title='Christ the Redeemer Experience', when='2026-01-11 10:56:21.780817')>


In [43]:
with get_session(engine) as session:
    memories = session.query(cultpass.UserMemory).all()
    for memory in memories:
        print(f"User ID: {memory.user_id}, Content: {memory.content}")

User ID: a4ab87, Content: Prefers outdoor activities
User ID: f556c0, Content: Interested in historical tours
User ID: a4ab87, Content: Allergic to peanuts
